In [2]:
import numpy as np
import pickle
import pandas as pd
from capacity.utils import *
import json

In [11]:
CIFAR100_DIR = '/mnt/home/cchou/ceph/Data/cifar100_train_processed'
MODELS_PATH = '/mnt/home/cchou/ceph/Capstone/CIFAR100_models/'
OUT_DIR = '/mnt/home/cchou/ceph/Capstone/CIFAR100_Dataframes_new/'
idx = pd.IndexSlice

with open("top_50_lr001.json") as json_file:
    top50_json = json.load(json_file)

In [4]:
epoch = 1
filename = OUT_DIR+'epoch'+str(epoch)+'.pkl'

In [5]:
with open(filename, 'rb') as f:
    df = pickle.load(f)

In [6]:
columns = []
for i_layer in range(len(df)):
    XtotT = df.loc[i_layer]['X_projected']
    columns += [(i_layer, i) for i in range(XtotT[0].shape[0])]

In [7]:
df_standard = pd.DataFrame()
num_layer = 7
num_class = 100
num_stim = 500
num_unit = 2000
df_new_list = []
for i_class in range(num_class):
    X = []
    iclass_top_bool = []
    for image_number in range(500):
        if image_number in top50_json[str(i_class)]:
            iclass_top_bool.append('top10_percent')
        else:
            iclass_top_bool.append('top')
    for i_layer in range(num_layer):
        X.append(df.loc[i_layer]['X_projected'][i_class].T)
    tuples = [(i_class, i, iclass_top_bool[i], 0) for i in range(num_stim)]
    index = pd.MultiIndex.from_tuples(tuples, names=['class', 'stimulus', 'type', 'repetition'])
    df_new = pd.DataFrame(np.array(X).transpose((1,0,2)).reshape(num_stim,num_layer*num_unit), index=index)
    df_new.columns = pd.MultiIndex.from_tuples(columns, names=['layer','unit'])
    df_new_list.append(df_new)
df_standard = pd.concat(df_new_list)

In [10]:
df_standard

layer                                  0                                \
unit                                0         1         2         3      
class stimulus type repetition                                           
0     0        top  0           1.276569 -0.722899  0.178733  0.638034   
      1        top  0           0.914976 -0.174200 -0.163422 -0.139505   
      2        top  0           0.382797  0.611522  0.961027 -0.060677   
      3        top  0           0.504028  0.077695  0.384816  0.699005   
      4        top  0           0.876069  1.189166  0.576951 -0.183255   
...                                  ...       ...       ...       ...   
99    495      top  0          -0.842246 -0.654935  0.790719 -3.438973   
      496      top  0           0.027976 -0.385947  0.441272 -1.389793   
      497      top  0          -0.429514  0.271708 -0.392003 -2.227589   
      498      top  0           0.400594  0.214798  0.209248 -2.723393   
      499      top  0           1.148362 -1.021067  0.261625 -2.899272   

layer                                                                   \
unit                                4         5         6         7      
class stimulus type repetition                                           
0     0        top  0          -2.022904  0.977287  0.728718  0.020204   
      1        top  0          -1.384757 -0.752696 -0.212502  0.778540   
      2        top  0          -0.762052 -0.168705 -0.496514  0.507706   
      3        top  0           0.016231 -1.247941 -1.478844  1.329019   
      4        top  0           0.504487 -0.824868 -1.253679  0.699173   
...                                  ...       ...       ...       ...   
99    495      top  0          -0.266781 -3.459869 -0.398666  1.290589   
      496      top  0          -4.297003 -4.298337 -1.073459  1.812608   
      497      top  0          -4.334067 -3.890911 -1.572555  0.935162   
      498      top  0          -4.056299 -3.388549 -2.802477  0.191312   
      499      top  0          -3.801254 -2.404270 -3.112453  0.675271   

layer                                               ...         6            \
unit                                8         9     ...      1990      1991   
class stimulus type repetition                      ...                       
0     0        top  0          -1.849872 -1.490902  ...  0.455694 -0.384720   
      1        top  0          -2.845151 -0.213412  ...  0.148648 -0.947033   
      2        top  0          -2.085694  0.026099  ...  0.367006 -1.166371   
      3        top  0          -2.249363  0.108006  ...  0.563043 -0.976051   
      4        top  0          -1.593066 -1.159302  ...  0.393426  0.080069   
...                                  ...       ...  ...       ...       ...   
99    495      top  0           0.361842 -1.494423  ... -0.117549  0.469864   
      496      top  0          -1.573951 -0.032806  ... -0.284901  0.331589   
      497      top  0          -1.768235  0.721082  ... -0.637763  0.825772   
      498      top  0          -1.349560  2.071727  ... -0.806833  0.949036   
      499      top  0          -0.962574  2.378397  ... -0.407465  0.433912   

layer                                                                   \
unit                                1992      1993      1994      1995   
class stimulus type repetition                                           
0     0        top  0           0.229209  0.043235 -0.173025 -0.113252   
      1        top  0           0.352653  0.156711 -0.159037  0.163393   
      2        top  0           0.383022  0.267774 -0.112457  0.470472   
      3        top  0           0.227756 -0.182775 -0.069957  0.496885   
      4        top  0          -0.048045 -0.132784  0.070575 -0.351869   
...                                  ...       ...       ...       ...   
99    495      top  0          -0.295473  0.891550  0.444769  0.084180   
      496      top  0          -0.180374  0.576000  0.958662  0.469645   
     

# Capacity Analysis

In [55]:
# Step 1: define manifolds
class_list = [i for i in range(40)]

In [ ]:
df_list = []
for layer_id in range(7):
    XtotT = [np.array(df_standard.loc[idx[class_id,0:39],layer_id]).T for class_id in class_list]
    result = manifold_analysis_all(XtotT)
    
    # Save results into a dataframe
    filtered_result = {key: value for key, value in result.items() if not isinstance(value, np.ndarray)}
    tuples = (layer_id,)
    index = pd.MultiIndex.from_tuples([tuples], names=['layer'])
    df_new = pd.DataFrame(filtered_result, index=index)
    df_list.append(df_new)
df_result = df.concat(df_list)

In [ ]:
df_result

In [13]:
lr = 0.001
epoch = 14
with open(f'{OUT_DIR}/LR_{lr}/epoch{epoch}.pkl', 'rb') as f:
    df = pickle.load(f)

In [14]:
df

,epoch,layer,X_projected
0,14,0,"[[[-0.25174270410045435, -0.49986150657693, -0..."
1,14,1,"[[[1.5414005541482205, 0.3146194619455442, 0.2..."
2,14,2,"[[[-0.2591719593438581, 0.010537586913302874, ..."
3,14,3,"[[[-1.0734681184626065, -1.2863588260016485, -..."
4,14,4,"[[[0.7266715190530344, 0.45642674353526475, 0...."
5,14,5,"[[[1.2017094769182077, 0.2906038360620481, 0.0..."
6,14,6,"[[[1.3154948416579013, 1.219846429161147, 1.69..."
